In [1]:
from math import sqrt  
import numpy as np     
from sklearn import datasets 
from sklearn.model_selection import train_test_split  

### Data

In [2]:
import pandas as pd

congressFeatureNames = ['party']
for i in range(16):
    congressFeatureNames.append('vote-'+str(i))

congressData = pd.read_csv('congress.data', header=None, names=congressFeatureNames )
mushroomFeatureNames = ['edible', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 
                'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 
                'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 
                'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']
    
mushroomData = pd.read_csv('agaricus-lepiota.data', header=None, names=mushroomFeatureNames)
len(mushroomData)

8124

In [3]:
class Node:
    def __init__(self, leaf, attr):
        self.isLeaf = leaf  # leaf node or not
        self.attr = attr    # if it's a leaf, what's the class label associated with it; 
                            # if not, which attribute to split on
        self.children = {}  # child nodes

In [4]:
def printTree(node, indent =0, val =0):
    if (node == None): 
        print('!!empty node!!')
        return
    if (node.isLeaf): # if it's a leaf node, print the class label
        for i in range(indent): # indent level
            print("|   ", end="")
        print("|--- class: ", node.attr)
    else: # for non-leaf nodes, we need to loop through our children and print each of them
        for val, child in node.children.items():
            for i in range(indent): # indent level
                print("|   ", end="")
            print("|--- ", node.attr, " = ", val) # print which attribute/value child is associated with
            printTree(child, indent+1, val) # print the sub-tree for child node

In [5]:
def testMyTree(data, trainFrac =0.7, seed =0):
    shuffled = data.sample(frac=1, random_state=seed) # shuffle examples randomly
    trainCount = int(len(data) * trainFrac) 
    train = shuffled[:trainCount] # training examples
    test = shuffled[trainCount:] # rest are for testing
    root = buildTree(train) 
    acc = testTree(root, test) 
    print("Test set accuracy: ", acc) 
    printTree(root) 

# Tree Building 
For each node, split on a feature

In [6]:
# split examples for a feature
# assume examples are in a pandas dataframe
def split(examples, attr):
    if isinstance(examples, pd.core.frame.DataFrame):
        if examples.shape[0] <= 1:#one example row then don't split it
            return examples
        else:
            sets = []
            values = np.unique(examples[attr])#list of unique values of attribute
            for value in values:#loop through values
                #append subset of examples with attribute = value
                sets.append(examples[examples[attr] == value])
            return sets
    else:
        print("examples should be a pandas dataframe")

In [7]:
#tests for split function
uniformData = pd.read_csv('uniform.data', header=None, names=['label', 'feature1'])
evenSplitData = pd.read_csv('even.data', header=None, names=['label', 'feature1'])
print(split(uniformData, 'feature1'))
print(split(evenSplitData, 'feature1'))
s = split(evenSplitData, 'feature1')
print(len(s[0].values))
for dataframe in s:
    for entry in dataframe.values:
        print(entry)
        
print(evenSplitData.columns[0])

oddData = pd.DataFrame({'label': ['red', 'red', 'red', 'red', 'blue', 'blue', 'blue'], 'feature1': ['y', 'y', 'n', 'n', 'y', 'y', 'y']})
print(f"oddData.iloc[0]: {oddData.iloc[0]}")
threeLabels = pd.DataFrame({'label': ['red', 'red', 'red', 'red', 'green', 'green', 'blue', 'blue', 'blue'], 'feature1': ['y', 'y', 'y', 'y', 'm', 'm', 'n', 'n', 'n']})
print(f"threeLabels.iloc[5]: {threeLabels.iloc[5]}")
print(f"threeLabels.iloc[8]: {threeLabels.iloc[8]}")
identity = pd.DataFrame({'label': ['red'], 'feature1': 'y'})#should always classify one example as red. 
print(f"df.shape[0] {identity.shape[0]}")
print(f"identity.iloc[0]: {identity.iloc[0]}")
twoFeatures = pd.DataFrame({'label': ['red', 'red', 'blue'], 'yesNo': ['y', 'n', 'n'], 'zeroOne': [0, 0, 1]})#zeroOne is more helpful than yes/no
print(f"twoFeatures.iloc[1]: {twoFeatures.iloc[1]}")
print(f"twoFeatures.iloc[2]: {twoFeatures.iloc[2]}")

print(f"split(oddData, 'feature1')\n{split(oddData, 'feature1')}")
#expect {'label': ['red', 'red', 'blue', 'blue', 'blue'], 'feature1': ['y', 'y' 'y ', 'y', 'y']}, {'label': ['red', 'red'], 'feature1': ['n', 'n']}
print(f"split(threeLabels, 'feature1')\n{split(threeLabels, 'feature1')}")
print(f"split(identity, 'feature1')\n{split(identity, 'feature1')}")
print(f"split(twoFeatures, 'feature1')\n{split(twoFeatures, 'yesNo')}")
print(f"split(twoFeatures, 'zeroOne')\n{split(twoFeatures, 'zeroOne')}")

[  label feature1
0   red        y
1   red        y
2   red        y
3   red        y]
[  label feature1
2  blue        n
3  blue        n,   label feature1
0   red        y
1   red        y]
2
['blue' 'n']
['blue' 'n']
['red' 'y']
['red' 'y']
label
oddData.iloc[0]: label       red
feature1      y
Name: 0, dtype: object
threeLabels.iloc[5]: label       green
feature1        m
Name: 5, dtype: object
threeLabels.iloc[8]: label       blue
feature1       n
Name: 8, dtype: object
df.shape[0] 1
identity.iloc[0]: label       red
feature1      y
Name: 0, dtype: object
twoFeatures.iloc[1]: label      red
yesNo        n
zeroOne      0
Name: 1, dtype: object
twoFeatures.iloc[2]: label      blue
yesNo         n
zeroOne       1
Name: 2, dtype: object
split(oddData, 'feature1')
[  label feature1
2   red        n
3   red        n,   label feature1
0   red        y
1   red        y
4  blue        y
5  blue        y
6  blue        y]
split(threeLabels, 'feature1')
[   label feature1
4  green        m
5

In [8]:
# entropy of a collections of examples with respect to a class
def entropy(examples, classAttr):
    if isinstance(examples, pd.core.frame.DataFrame):
        total = 0
        if len(examples.values) > 0:
            if examples.shape[0] <= 1:#otherwise it will iterate through strings
                return 0.0
            subsets = split(examples, classAttr)
            n = len(examples.values)
            for frame in subsets:
                probability_Si = len(frame.values)/n
                total += probability_Si*(np.log2(probability_Si))
        else:
            print("examples is empty")
        return -1*total
    else:
        print("examples should be a pandas dataframe")

In [9]:
# tests for entropy()
print(entropy(uniformData, 'label')) # should be 0
print(entropy(evenSplitData, 'label')) # should be 1
uniformData.value_counts().idxmax()
evenSplitData.value_counts().idxmax()
testSet = pd.DataFrame({'label': ['red', 'red', 'blue'], 'feature1': ['y', 'y', 'n']}).value_counts().idxmax()
pd.DataFrame({'label': ['red', 'red', 'red'], 'feature1': ['y', 'y', 'n']}).value_counts().idxmax()

print(f"entropy(oddData, 'feature1')\n{entropy(oddData, 'feature1')}")
print(f"entropy(threeLabels, 'feature1')\n{entropy(threeLabels, 'feature1')}")
print(f"entropy(identity, 'feature1')\n{entropy(identity, 'feature1')}")
print(f"entropy(twoFeatures, 'feature1')\n{entropy(twoFeatures, 'yesNo')}")
print(f"entropy(twoFeatures, 'zeroOne')\n{entropy(twoFeatures, 'zeroOne')}")

-0.0
1.0
entropy(oddData, 'feature1')
0.863120568566631
entropy(threeLabels, 'feature1')
1.5304930567574826
entropy(identity, 'feature1')
0.0
entropy(twoFeatures, 'feature1')
0.9182958340544896
entropy(twoFeatures, 'zeroOne')
0.9182958340544896


In [10]:
# information gain with respect to an attribute
def informationGain(examples, attr, classAttr): 
    setEntropy = 0
    if len(examples.values) > 0:
        if examples.shape[0] <= 1:#otherwise it will iterate through strings
                return 0.0
        subsets = split(examples, attr)
        n = len(examples.values)
        for frame in subsets:
            probability_Si = len(frame.values)/n
            setEntropy += entropy(frame, classAttr) * probability_Si
    return entropy(examples, classAttr) - setEntropy
print(informationGain(uniformData, 'feature1', 'label')) 
print(informationGain(evenSplitData, 'feature1', 'label')) #test a bit more. 
#best to test: tiny lil examples. 

-0.0
1.0


In [11]:
# information gain tests
print(f"informationGain(oddData, 'feature1', 'label')\n{informationGain(oddData, 'feature1', 'label')}")
print(f"informationGain(threeLabels, 'feature1', 'label')\n{informationGain(threeLabels, 'feature1', 'label')}")
print(f"informationGain(identity, 'feature1', 'label')\n{informationGain(identity, 'feature1', 'label')}")
print(f"informationGain(twoFeatures, 'feature1', 'label')\n{informationGain(twoFeatures, 'yesNo', 'label')}")
print(f"informationGain(twoFeatures, 'zeroOne', 'label')\n{informationGain(twoFeatures, 'zeroOne', 'label')}")

informationGain(oddData, 'feature1', 'label')
0.2916919971380597
informationGain(threeLabels, 'feature1', 'label')
1.5304930567574826
informationGain(identity, 'feature1', 'label')
0.0
informationGain(twoFeatures, 'feature1', 'label')
0.2516291673878229
informationGain(twoFeatures, 'zeroOne', 'label')
0.9182958340544896


In [12]:
# see which value of the attribute attr has a plurality
def plurality(examples, attr):
    if examples.shape[0] <= 1:#otherwise it will bug out
                return examples[attr][0]
    subsetList = split(examples, attr)#split on attributes. 
    maxLength = 0
    maxLengthValue = 0
    for subset in subsetList:
        subsetLen = len(subset)
        if subsetLen > maxLength:
            maxLength = subsetLen
            maxLengthValue = subset[attr].iloc[0]#attribute of first entry in subset
    if maxLengthValue == 0:
        print('no subset found')
    return maxLengthValue

In [13]:
# plurality tests
print(f"plurality(oddData, 'feature1')\n{plurality(oddData, 'feature1')}")
print(f"plurality(threeLabels, 'feature1')\n{plurality(threeLabels, 'feature1')}")
print(f"plurality(identity, 'feature1')\n{plurality(identity, 'feature1')}")
print(f"plurality(twoFeatures, 'feature1')\n{plurality(twoFeatures, 'yesNo')}")
print(f"plurality(twoFeatures, 'zeroOne')\n{plurality(twoFeatures, 'zeroOne')}")

plurality(oddData, 'feature1')
y
plurality(threeLabels, 'feature1')
y
plurality(identity, 'feature1')
y
plurality(twoFeatures, 'feature1')
n
no subset found
plurality(twoFeatures, 'zeroOne')
0


In [14]:
# create a tree node by splitting on the attribute with the highest information gain
# takes in set of examples for current node. 
def buildTree(examples):
    cols = examples.columns
    if examples.shape[0] <= 1:#otherwise it will iterate through a string
        return Node(True, examples[cols[0]].iloc[0])#value for label
    if len(cols) > 1:#splitting on all features
        maxIgain = -1
        tempIgain = 0
        maxCol = -1
        for i in range(1, len(cols)):#label is always at index 0
            col = cols[i]
            tempIgain = informationGain(examples, col, examples.columns[0])#not necessairily 0
            if tempIgain > maxIgain:
                maxIgain = tempIgain
                maxCol = col
        splitColumn = split(examples, maxCol)
        treeNode = Node(False, maxCol)
        for col in splitColumn:
            key = col[maxCol].iloc[0]#value for attribute
            col = col.drop(columns = maxCol, axis = 1)
            newNode = buildTree(col)
            treeNode.children[key] = newNode
        return treeNode       
    else:
        #pandas: select examples with some value in column. 
        #df['item'].value_counts().idxmax()
        return Node(True, plurality(examples, cols[0]))

In [15]:
test1 = evenSplitData
printTree(buildTree(test1))
test2 = pd.DataFrame({'label': ['red', 'red', 'red'], 'feature1': ['y', 'y', 'n']})
printTree(buildTree(test2))

print(f"buildTree(oddData)")
oddDataDecisionTree = buildTree(oddData)
printTree(oddDataDecisionTree)
print(f"buildTree(threeLabels)")
threeLabelsDecisionTree = buildTree(threeLabels)
printTree(threeLabelsDecisionTree)
print(f"buildTree(identity)")
identityDecisionTree = buildTree(identity)
printTree(identityDecisionTree)
print(f"buildTree(twoFeatures)")
twoFeaturesDecisionTree = buildTree(twoFeatures)
printTree(twoFeaturesDecisionTree)

|---  feature1  =  n
|   |--- class:  blue
|---  feature1  =  y
|   |--- class:  red
|---  feature1  =  n
|   |--- class:  red
|---  feature1  =  y
|   |--- class:  red
buildTree(oddData)
|---  feature1  =  n
|   |--- class:  red
|---  feature1  =  y
|   |--- class:  blue
buildTree(threeLabels)
|---  feature1  =  m
|   |--- class:  green
|---  feature1  =  n
|   |--- class:  blue
|---  feature1  =  y
|   |--- class:  red
buildTree(identity)
|--- class:  red
buildTree(twoFeatures)
|---  zeroOne  =  0
|   |---  yesNo  =  n
|   |   |--- class:  red
|   |---  yesNo  =  y
|   |   |--- class:  red
|---  zeroOne  =  1
|   |--- class:  blue


In [16]:
import random   

#classify example using tree
def classify(node, example):
    if node.isLeaf:
        return node.attr #attribute has answer. 
    else:
        nodeKeys = list(node.children.keys())
        newNodeKey = example[node.attr]
        if newNodeKey in nodeKeys:
            return classify(node.children[newNodeKey], example)#node child with same value as attribute as example does
        else:#feature we split on not in training set, return a random key
            return classify(node.children[random.choice(nodeKeys)], example)#node child with same value as attribute as example does

In [17]:
print(f"oddData.iloc[0]: {oddData.iloc[0]}")
print(classify(oddDataDecisionTree, oddData.iloc[0]))
print(f"threeLabels.iloc[5]: {threeLabels.iloc[5]}")
print(classify(threeLabelsDecisionTree, threeLabels.iloc[5]))
print(f"threeLabels.iloc[8]: {threeLabels.iloc[8]}")
print(classify(threeLabelsDecisionTree, threeLabels.iloc[8]))
print(f"identity.iloc[0]: {identity.iloc[0]}")
print(classify(identityDecisionTree, identity.iloc[0]))
print(f"twoFeatures.iloc[1]: {twoFeatures.iloc[1]}")
print(classify(twoFeaturesDecisionTree, twoFeatures.iloc[1]))
print(f"twoFeatures.iloc[2]: {twoFeatures.iloc[2]}")
print(classify(twoFeaturesDecisionTree, twoFeatures.iloc[2]))

oddData.iloc[0]: label       red
feature1      y
Name: 0, dtype: object
blue
threeLabels.iloc[5]: label       green
feature1        m
Name: 5, dtype: object
green
threeLabels.iloc[8]: label       blue
feature1       n
Name: 8, dtype: object
blue
identity.iloc[0]: label       red
feature1      y
Name: 0, dtype: object
red
twoFeatures.iloc[1]: label      red
yesNo        n
zeroOne      0
Name: 1, dtype: object
red
twoFeatures.iloc[2]: label      blue
yesNo         n
zeroOne       1
Name: 2, dtype: object
blue


In [18]:
# finds the accuracy of the tree on that test set
def testTree(root, testSet):
    classifier = testSet.columns[0]
    accuracySum = 0
    for index, data in testSet.iterrows():#loop through examples, check if we got the right class
        label = classify(root, data)
        if label == data[classifier]:
            accuracySum += 1
    return accuracySum/len(testSet)

In [19]:
print("testMyTree(evenSplitData)")
testMyTree(evenSplitData)
print("testMyTree(oddData)")
testMyTree(oddData)
print("testMyTree(threeLabels)")
testMyTree(threeLabels)
#print("testMyTree(evenSplitData)")
#testMyTree(identity)
print("testMyTree(twoFeatures)")
testMyTree(twoFeatures)

print("Tree on Mushroom data:")
testMyTree(mushroomData) 
print("Tree on Congress data:")
testMyTree(congressData)
print("Done")

DT on evensplitdata:
trainAndTestDT(evenSplitData)
Test set accuracy:  0.0
|---  feature1  =  n
|   |--- class:  blue
trainAndTestDT(oddData)
Test set accuracy:  0.6666666666666666
|---  feature1  =  n
|   |--- class:  red
|---  feature1  =  y
|   |--- class:  blue
trainAndTestDT(threeLabels)
Test set accuracy:  1.0
|---  feature1  =  m
|   |--- class:  green
|---  feature1  =  n
|   |--- class:  blue
|---  feature1  =  y
|   |--- class:  red
trainAndTestDT(twoFeatures)
Test set accuracy:  1.0
|---  zeroOne  =  0
|   |--- class:  red
|---  zeroOne  =  1
|   |--- class:  blue
DT on Mushroom data:
Test set accuracy:  1.0
|---  odor  =  a
|   |---  cap-shape  =  b
|   |   |---  cap-surface  =  s
|   |   |   |---  cap-color  =  w
|   |   |   |   |---  bruises  =  t
|   |   |   |   |   |---  gill-attachment  =  f
|   |   |   |   |   |   |---  gill-spacing  =  c
|   |   |   |   |   |   |   |---  gill-size  =  b
|   |   |   |   |   |   |   |   |---  gill-color  =  g
|   |   |   |   |   |   | 